<a href="https://colab.research.google.com/github/psgrigoletti/curso-pmf/blob/main/Pablo_Final_de_An%C3%A1lise_Fundamentalista_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Python para Mercado Financeiro, Módulo 7 - Análise Fundamentalista em Python**

**Autor:** Pablo Souza Grigoletti

**E-mail:** psgrigoletti@gmail.com



#1. Instalando e importando as bibliotecas necessárias

In [299]:
# Instalação das bibliotecas
!pip install Plotly
!pip install investpy
!pip install xlsxwriter

# Import das bibliotecas
import pandas as pd
import numpy as np
import investpy

import requests
import zipfile
import xlsxwriter

import io
import os

import plotly
import plotly.graph_objects as go

import time
from datetime import date

from multiprocessing import Pool

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# 2. Buscando os dados das empresas listadas

Vamos extrair apenas o documento descritivo das companhias de capital aberto extraído da CVM. Com isso, poderemos filtrar as empresas de acordo com a característica que queremos e pegar os códigos de empresa.

##2.1. Baixando os dados

In [300]:
# r = requests.get('http://dados.cvm.gov.br/dados/CIA_ABERTA/CAD/DADOS/cad_cia_aberta.csv')
# lines = [i.strip().split(';') for i in r.text.split('\n')]
# df = pd.DataFrame(lines[1:], columns = lines[0])
# df.head()

df_dados_empresas = pd.read_csv('http://dados.cvm.gov.br/dados/CIA_ABERTA/CAD/DADOS/cad_cia_aberta.csv', sep=";", encoding="ISO-8859-1")
df_dados_empresas.head()

,CNPJ_CIA,DENOM_SOCIAL,DENOM_COMERC,DT_REG,DT_CONST,DT_CANCEL,MOTIVO_CANCEL,SIT,DT_INI_SIT,CD_CVM,...,UF_RESP,PAIS_RESP,CEP_RESP,DDD_TEL_RESP,TEL_RESP,DDD_FAX_RESP,FAX_RESP,EMAIL_RESP,CNPJ_AUDITOR,AUDITOR
0,08.773.135/0001-00,2W ENERGIA S.A.,BANCO BESA SA,2020-10-29,2007-03-23,NaN,NaN,ATIVO,2020-10-29,25224,...,SP,NaN,4707000.0,11,39579400.0,NaN,NaN,ri@2wenergia.com.br,61.366.936/0001-25,ERNST & YOUNG AUDITORES INDEPENDENTES S/S LTDA.
1,11.396.633/0001-87,3A COMPANHIA SECURITIZADORA,TRIPLO A COMPANHIA SECURITIZADORA,2010-03-08,2009-11-03,2015-12-18,Cancelamento Voluntário - IN CVM 480/09,CANCELADA,2015-12-18,21954,...,RJ,NaN,20020000.0,21,22338867.0,21.0,22338867.0,juridico@triploasec.com.br,60.525.706/0001-07,MOORE STEPHENS LIMA LUCCHESI AUDITORES INDEPEN...
2,12.091.809/0001-55,3R PETROLEUM OLÉO E GÁS S.A.,NaN,2020-11-09,2010-06-08,NaN,NaN,ATIVO,2020-11-09,25291,...,RJ,NaN,22250180.0,21,21455555.0,0.0,0.0,ri@3rpetroleum.com.br,57.755.217/0001-29,KPMG AUDITORES INDEPENDENTES LTDA.
3,01.547.749/0001-16,521 PARTICIPAÇOES S.A. - EM LIQUIDAÇÃO EXTRAJU...,521 PARTICIPAÇÕES S/A,1997-07-11,1996-07-30,NaN,NaN,ATIVO,1997-07-11,16330,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.596.945/0001-83,BAKER TILLY 4PARTNERS AUDITORES INDEPENDENTES S/S
4,01.851.771/0001-55,524 PARTICIPAÇOES SA,524 PARTICIPACOES SA,1997-05-30,1997-04-02,NaN,NaN,ATIVO,1997-05-30,16284,...,RJ,NaN,20020010.0,21,38043700.0,0.0,0.0,gar@opportunity.com.br,40.262.602/0001-31,"BKR - LOPES, MACHADO AUDITORES"


##2.2. Filtrando o DataFrame

Procurando por empresas específicas, que tenham uma característica em comum na sua denominação comercial.

In [301]:
situacao_interesse = 'ATIVO'
tipo_mercado_interesse = 'BOLSA'

df_dados_empresas_filtradas = df_dados_empresas
df_dados_empresas_filtradas = df_dados_empresas_filtradas[(df_dados_empresas_filtradas['SIT'] == situacao_interesse) & (df_dados_empresas_filtradas['TP_MERC'] == tipo_mercado_interesse)]

# TODO: trocar o código abaixo e encontrar uma forma melhor de filtrar os ativos...
cnpjs_interesse = ['05.878.397/0001-32', '07.816.890/0001-53', '51.218.147/0001-93', '06.977.751/0001-49', '06.977.745/0001-91']
denominacoes_interesse = ['MULTIPLAN', 'MALL', 'IGUATEMI', 'SHOPP']
df_dados_empresas_filtradas = df_dados_empresas_filtradas[df_dados_empresas_filtradas['CNPJ_CIA'].isin(cnpjs_interesse)]
df_dados_empresas_filtradas = df_dados_empresas_filtradas[df_dados_empresas_filtradas.DENOM_SOCIAL.str.contains('|'.join(denominacoes_interesse), na=False)]
df_dados_empresas_filtradas

# Precisamos obter os códigos CVM das empresas que queremos obter.
# Uma vez que sabemos os códigos a partir da tabela anterior, vamos transformar essa coluna em uma lista.
# Já ajustado para ficar com 6 dígitos, preenchidos com zero a esquerda
codigos_cvm = list(df_dados_empresas_filtradas['CD_CVM'])
codigos_cvm = list(map(lambda i: str(i).zfill(6), codigos_cvm))
codigos_cvm

['022357', '019909', '020494', '020982']

# 3. Extraindo as suas demonstrações financeiras

## 3.1. Descrição

Quais as demonstrações que você precisa? O conjunto de dados disponibiliza as seguintes demonstrações financeiras entregues nos últimos cinco anos:
- Balanço Patrimonial Ativo (BPA)
- Balanço Patrimonial Passivo (BPP)
- Demonstração de Fluxo de Caixa - Método Direto (DFC-MD)
- Demonstração de Fluxo de Caixa - Método Indireto (DFC-MI)
- Demonstração das Mutações do Patrimônio Líquido (DMPL)
- Demonstração de Resultado Abrangente (DRA)
- Demonstração de Resultado (DRE)
- Demonstração de Valor Adicionado (DVA)

**OBSERVAÇÃO: Diferença entre balanço patrimonial Consolidado e Individual:**

Nas demonstrações financeiras (DFs) onde encontramos o balanço patrimonial, você vai notar que há duas seções: as DFs individuais e DFs consolidadas. Dentro de cada uma, há diferentes balanços patrimoniais.

A diferença é que o balanço consolidado traz informações referentes à empresa principal e também às subsidiárias (controladas pela empresa principal). 

Já os números do balanço individual se referem somente à "empresa mãe".

Entre os investidores, o mais comum é utilizar as informações consolidadas na análise. Afinal, é a que dá a visão mais realista da posição financeira da companhia.

O balanço patrimonial individual pode ser considerado em casos em que as subsidiárias atuam em segmentos totalmente diferentes da empresa principal e não sofrem grandes intervenções de sua controladora.

##3.2. Baixando os arquivos e ajustando os arquivos (processamento em paralelo)

In [302]:
demonstrativos_interesse = ['BPA', 'DRE', 'BPP']

In [303]:
def retorna_ponteiro_arquivo_zip(url):
    try:
        req = requests.get(url)
        arquivo_zip = zipfile.ZipFile(io.BytesIO(req.content))
        print('Sucesso ao baixar o arquivo de ' + url)
    except:
        print("Erro ao buscar o arquivo de " + url)
        arquivo_zip = None
    return arquivo_zip

def atualiza_dict_df_demonstrativo(tipo, arquivo_zip, demonstrativo, ano, codigos_cvm, dict_df):
    if not arquivo_zip is None:
        nome_arquivo_csv = tipo + '_cia_aberta_' + str(demonstrativo) + '_con_' + str(ano) + '.csv' # CONSIDERANDO SOMENTE OS DF CONSOLIDADOS
        arquivo_csv = arquivo_zip.open(nome_arquivo_csv)
        linhas = arquivo_csv.readlines()
        linhas = [i.strip().decode('ISO-8859-1') for i in linhas]
        linhas = [i.split(';') for i in linhas]
        df = pd.DataFrame(linhas[1:], columns = linhas[0])
        df['VL_AJUSTADO'] = pd.to_numeric(df['VL_CONTA'])
        df = df[df['ORDEM_EXERC']=="ÚLTIMO"]

        for codigo_cvm in codigos_cvm:
            if not codigo_cvm + "|" + demonstrativo in dict_df:
                dict_df[codigo_cvm + "|" + demonstrativo] = df[df['CD_CVM']== str(codigo_cvm)]
            else:
                dict_df[codigo_cvm + "|" + demonstrativo] = pd.concat([dict_df[codigo_cvm + "|" + demonstrativo], 
                                                                       df[df['CD_CVM']== str(codigo_cvm)]])

def processa_arquivos_do_ano(ano):
    url_itr = 'http://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/ITR/DADOS/itr_cia_aberta_' + str(ano) + '.zip'
    url_dfp = 'https://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/DFP/DADOS/dfp_cia_aberta_' + str(ano) + '.zip'
    arquivo_zip_itr = retorna_ponteiro_arquivo_zip(url_itr)
    arquivo_zip_dfp = retorna_ponteiro_arquivo_zip(url_dfp)

    dict_df = {}

    #for i in range(0, len(codigos_cvm)):
        #codigo_cvm = codigos_cvm[i]
        # writer = pd.ExcelWriter(f'demonstrativos_empresa_{str(codigo_cvm)}_ano_{ano}.xlsx', engine='xlsxwriter')
    for demonstrativo in demonstrativos_interesse:
        
        atualiza_dict_df_demonstrativo("itr", arquivo_zip_itr, demonstrativo, ano, codigos_cvm, dict_df)
        atualiza_dict_df_demonstrativo("dfp", arquivo_zip_dfp, demonstrativo, ano, codigos_cvm, dict_df)
        #df = pd.concat([df_itr, df_dfp])

        #df.to_excel(writer, sheet_name=demonstrativo, encoding= 'ISO-8859-1')
        print(f'Dados (de todas as empresas de interesse) do demonstrativo {demonstrativo} para o ano {ano} foram armazenados.')
        #writer.save()
        #print(f'Arquivo Excel com os demonstrativos da empresa {codigo_cvm} para o ano {ano} já exportado.\n')

    for i in range(0, len(codigos_cvm)):
        codigo_cvm = codigos_cvm[i]
        writer = pd.ExcelWriter(f'demonstrativos_empresa_{str(codigo_cvm)}_ano_{ano}.xlsx', engine='xlsxwriter')        

        for demonstrativo in demonstrativos_interesse:
            dict_df[codigo_cvm + "|" + demonstrativo].to_excel(writer, sheet_name=demonstrativo, encoding= 'ISO-8859-1')
        
        writer.save()
        print(f'Arquivo Excel com os demonstrativos da empresa {codigo_cvm} para o ano {ano} já exportado.')    

start_time = time.time()
anos_interesse = [2019, 2020, 2021, 2022]

with Pool() as pool:
    pool.map(processa_arquivos_do_ano, anos_interesse)
    
print("O tempo de execução foi de %s segundos" % (time.time() - start_time))

Sucesso ao baixar o arquivo de http://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/ITR/DADOS/itr_cia_aberta_2019.zip
Sucesso ao baixar o arquivo de http://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/ITR/DADOS/itr_cia_aberta_2020.zip
Sucesso ao baixar o arquivo de https://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/DFP/DADOS/dfp_cia_aberta_2019.zip
Sucesso ao baixar o arquivo de https://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/DFP/DADOS/dfp_cia_aberta_2020.zip
Dados (de todas as empresas de interesse) do demonstrativo BPA para o ano 2019 foram armazenados.
Dados (de todas as empresas de interesse) do demonstrativo BPA para o ano 2020 foram armazenados.
Dados (de todas as empresas de interesse) do demonstrativo DRE para o ano 2019 foram armazenados.
Dados (de todas as empresas de interesse) do demonstrativo DRE para o ano 2020 foram armazenados.
Dados (de todas as empresas de interesse) do demonstrativo BPP para o ano 2019 foram armazenados.
Arquivo Excel com os demonstrativos da empresa 022357 para o ano 201

#4. Transformando os dados

##4.1. Verificando os arquivos XLSX gerados

In [304]:
# Capturar os arquivos que estão no diretório criado na etapa do webscrapping
caminho = os.getcwd()
arquivos = os.listdir(caminho)
# arquivos

# Juntamos todos esses xslxs em uma lista
arquivos_xlsx = [f for f in arquivos if f[-4:] == 'xlsx']
arquivos_xlsx

['demonstrativos_empresa_022357_ano_2019.xlsx',
 'demonstrativos_empresa_019909_ano_2021.xlsx',
 'demonstrativos_empresa_020982_ano_2020.xlsx',
 'demonstrativos_empresa_019909_ano_2022.xlsx',
 'demonstrativos_empresa_022357_ano_2021.xlsx',
 'demonstrativos_empresa_019909_ano_2020.xlsx',
 'demonstrativos_empresa_020494_ano_2022.xlsx',
 'demonstrativos_empresa_020982_ano_2021.xlsx',
 'demonstrativos_empresa_020494_ano_2019.xlsx',
 'demonstrativos_empresa_020494_ano_2020.xlsx',
 'demonstrativos_empresa_020494_ano_2021.xlsx',
 'demonstrativos_empresa_020982_ano_2022.xlsx',
 'demonstrativos_empresa_020982_ano_2019.xlsx',
 'demonstrativos_empresa_019909_ano_2019.xlsx',
 'demonstrativos_empresa_022357_ano_2020.xlsx',
 'demonstrativos_empresa_022357_ano_2022.xlsx']

##4.2. Tabela DRE

In [305]:
def adicionar_dados_4_trimestre(dre):
    dre_infos_12_meses = dre[(dre['DT_INI_EXERC'].str[4:8] == '-01-') & (dre['DT_FIM_EXERC'].str[4:8] == '-12-')]
    dre_infos_9_meses = dre[(dre['DT_INI_EXERC'].str[4:8] == '-01-') & (dre['DT_FIM_EXERC'].str[4:8] == '-09-')]

    for index, row in dre_infos_12_meses.iterrows():
        codigo_cvm = row['CD_CVM']
        ordem = row['ORDEM_EXERC']
        codigo_conta = row['CD_CONTA']
        descricao_conta = row['DS_CONTA']
        valor_ajustado_12_meses = row['VL_AJUSTADO']
        nova_data_inicio = row['DT_INI_EXERC'][0:4] + '-10-01'

        try:
            valor_ajustado_9_meses = dre_infos_9_meses[(dre_infos_9_meses['CD_CONTA'] == codigo_conta) & 
                                                       (dre_infos_9_meses['ORDEM_EXERC'] == ordem)].iloc[0]['VL_AJUSTADO']
            #print(f"Achou valor para a conta {codigo_conta} {descricao_conta}")
            #print(f"Valor para os 9 meses: {valor_ajustado_9_meses}")
            #print(f"Valor para os 12 meses: {valor_ajustado_12_meses}")
            #print(f"Vai setar a nova data inicial para: {nova_data_inicio}\n\n")
            row_temp = row.copy()
            row_temp['VL_AJUSTADO'] = valor_ajustado_12_meses - valor_ajustado_9_meses
            row_temp['DT_INI_EXERC'] = nova_data_inicio
            dre = dre.append([row_temp])
        except:
            print(f"Não encontrou valor para a conta {codigo_conta} - '{descricao_conta}' para os 9 meses. Encontrou para os 12.")
    
    return dre

# Criar o dataframe DRE
dre_completo = pd.DataFrame()

# Juntar todos os arquivos xlsx em uma grande tabela DRE
for arquivo_xlsx in arquivos_xlsx:
    dre = pd.read_excel(arquivo_xlsx, sheet_name='DRE')
    dre = adicionar_dados_4_trimestre(dre)
    dre_completo = dre_completo.append(dre)

Não encontrou valor para a conta 3.02.03 - 'Custos dos imóveis vendidos' para os 9 meses. Encontrou para os 12.
Não encontrou valor para a conta 3.04.05.03 - 'Outras despesas operacionais' para os 9 meses. Encontrou para os 12.
Não encontrou valor para a conta 3.02.03 - 'Custos dos imóveis vendidos' para os 9 meses. Encontrou para os 12.
Não encontrou valor para a conta 3.04.05.03 - 'Outras despesas operacionais' para os 9 meses. Encontrou para os 12.
Não encontrou valor para a conta 3.02.03 - 'Custos dos imóveis vendidos' para os 9 meses. Encontrou para os 12.
Não encontrou valor para a conta 3.04.05.03 - 'Outras despesas operacionais' para os 9 meses. Encontrou para os 12.
Não encontrou valor para a conta 3.02.03 - 'Custos dos imóveis vendidos' para os 9 meses. Encontrou para os 12.
Não encontrou valor para a conta 3.04.05.03 - 'Outras despesas operacionais' para os 9 meses. Encontrou para os 12.


In [306]:
# Puxar uma lista com os nomes das empresas
lista_de_empresas = dre_completo['DENOM_CIA'].unique().tolist()
lista_de_empresas

['ALIANSCE SONAE SHOPPING CENTERS S.A.',
 'BR MALLS PARTICIPACOES S.A.',
 'MULTIPLAN - EMPREEND IMOBILIARIOS S.A.',
 'IGUATEMI EMPRESA DE SHOPPING CENTERS S.A']

In [307]:
#tickers = ['IGTA3', 'BRML3', 'MULT3', 'ALSO3'] # precisa estar na mesma ordem da lista de empresas

In [308]:
lista_de_contas = pd.DataFrame()
lista_de_contas["CONTA"] = dre_completo['CD_CONTA'] + " - " + dre_completo['DS_CONTA']
lista_de_contas["CONTA"].unique().tolist()

['3.01 - Receita de Venda de Bens e/ou Serviços',
 '3.02 - Custo dos Bens e/ou Serviços Vendidos',
 '3.03 - Resultado Bruto',
 '3.04 - Despesas/Receitas Operacionais',
 '3.04.01 - Despesas com Vendas',
 '3.04.02 - Despesas Gerais e Administrativas',
 '3.04.03 - Perdas pela Não Recuperabilidade de Ativos',
 '3.04.04 - Outras Receitas Operacionais',
 '3.04.04.01 - Outras Receitas Operacionais Líquidas',
 '3.04.05 - Outras Despesas Operacionais',
 '3.04.05.01 - Despesas Tributárias',
 '3.04.06 - Resultado de Equivalência Patrimonial',
 '3.05 - Resultado Antes do Resultado Financeiro e dos Tributos',
 '3.06 - Resultado Financeiro',
 '3.06.01 - Receitas Financeiras',
 '3.06.02 - Despesas Financeiras',
 '3.07 - Resultado Antes dos Tributos sobre o Lucro',
 '3.08 - Imposto de Renda e Contribuição Social sobre o Lucro',
 '3.08.01 - Corrente',
 '3.08.02 - Diferido',
 '3.09 - Resultado Líquido das Operações Continuadas',
 '3.10 - Resultado Líquido de Operações Descontinuadas',
 '3.10.01 - Lucro/

In [309]:
# Organizar a tabela em um esquema de tabela dinamica, onde as datas sejam labels de coluna
dre_completo = pd.pivot_table(dre_completo, index=['DENOM_CIA','DS_CONTA'], columns=['DT_INI_EXERC', 'DT_FIM_EXERC'],values=['VL_AJUSTADO'])
dre_completo.head()

VL_AJUSTADO  \
DT_INI_EXERC                                                                     2019-01-01   
DT_FIM_EXERC                                                                     2019-03-31   
DENOM_CIA                            DS_CONTA                                                 
ALIANSCE SONAE SHOPPING CENTERS S.A. Atribuído a Sócios Não Controladores           17822.0   
                                     Atribuído a Sócios da Empresa Controladora     29162.0   
                                     Corrente                                       -6305.0   
                                     Custo dos Bens e/ou Serviços Vendidos         -19510.0   
                                     Despesas Financeiras                               0.0   

                                                                                            \
DT_INI_EXERC                                                                                 
DT_FIM_EXERC                                                                    2019-06-30   
DENOM_CIA                            DS_CONTA                                                
ALIANSCE SONAE SHOPPING CENTERS S.A. Atribuído a Sócios Não Controladores          35121.0   
                                     Atribuído a Sócios da Empresa Controladora    63755.0   
                                     Corrente                                     -16474.0   
                                     Custo dos Bens e/ou Serviços Vendidos        -39448.0   
                                     Despesas Financeiras                         -36176.0   

                                                                                            \
DT_INI_EXERC                                                                                 
DT_FIM_EXERC                                                                    2019-09-30   
DENOM_CIA                            DS_CONTA                                                
ALIANSCE SONAE SHOPPING CENTERS S.A. Atribuído a Sócios Não Controladores          15461.0   
                                     Atribuído a Sócios da Empresa Controladora   -78594.0   
                                     Corrente                                     -24797.0   
                                     Custo dos Bens e/ou Serviços Vendidos       -123307.0   
                                     Despesas Financeiras                        -139229.0   

                                                                                            \
DT_INI_EXERC                                                                                 
DT_FIM_EXERC                                                                    2019-12-31   
DENOM_CIA                            DS_CONTA                                                
ALIANSCE SONAE SHOPPING CENTERS S.A. Atribuído a Sócios Não Controladores          34605.0   
                                     Atribuído a Sócios da Empresa Controladora    22027.0   
                                     Corrente                                     -41463.0   
                                     Custo dos Bens e/ou Serviços Vendidos       -200075.0   
                                     Despesas Financeiras                        -207886.0   

                                                                                            \
DT_INI_EXERC                                                                    2019-04-01   
DT_FIM_EXERC                                                                    2019-06-30   
DENOM_CIA                            DS_CONTA                                                
ALIANSCE SONAE SHOPPING CENTERS S.A. Atribuído a Sócios Não Controladores          17299.0   
                                     Atribuído a Sócios da Empresa Controladora    34593.0   
                                     Corrente                                     -10169.0   
                         

##4.3. Tabelas de Balanço Patrimonal Ativo (BPA) e Balanço Patrimonal Passivo (BPP)

Balanço patrimonial é um documento no qual as empresas divulgam números referentes a seus ativos, passivos e patrimônio líquido. 

Companhias de capital aberto, isto é, com ações negociadas na Bolsa de Valores, são obrigadas a divulgar o balanço patrimonial trimestralmente no Formulário de Informações Trimestrais (ITR) e anualmente no formulário de Demonstrações Financeiras Padronizadas (DFP).

Desse modo, qualquer pessoa pode conferir qual a situação da empresa em que investe ou considera investir em um trimestre específico, ou então no consolidado do ano.

Como o nome já entrega, as DFPs são padronizadas, o que significa que todas as empresas estruturam e classificam as informações da mesma maneira — inclusive os dados referentes ao balanço patrimonial.

Isso torna a experiência de quem analisa múltiplas empresas muito mais fácil, pois é possível encontrar rapidamente as informações desejadas e comparar os dados de diferentes companhias.

Os ITRs e DFPs dividem em dois grupos os números do balanço patrimonial:

###4.3.1. Balanço Patrimonial Ativo (BPA)

Um ativo é um bem ou direito da empresa. Um recurso que, em tese, é útil ou necessário para ajudá-la a obter benefícios econômicos futuros.

O dinheiro em caixa, por exemplo, é um ativo necessário para pagar os funcionários que executam as rotinas administrativas, financeiras e operacionais do negócio.

Mas há diversos tipos de ativos, e o balanço patrimonial os divide entre **ativo circulante** — os bens e direitos que têm maior liquidez, ou seja, podem ser convertidos em dinheiro com mais facilidade e rapidez — e **ativo não circulante** — com menor liquidez.

In [310]:
# Preparando o BPA
bpa_completo = pd.DataFrame()

for f in arquivos_xlsx:
    bpa = pd.read_excel(f, sheet_name='BPA')
    bpa_completo = bpa_completo.append(bpa)

In [311]:
lista_de_contas = pd.DataFrame()
lista_de_contas["CONTA"] = bpa_completo['CD_CONTA'] + " - " + bpa_completo['DS_CONTA']
lista_de_contas["CONTA"].unique().tolist()

['1 - Ativo Total',
 '1.01 - Ativo Circulante',
 '1.01.01 - Caixa e Equivalentes de Caixa',
 '1.01.02 - Aplicações Financeiras',
 '1.01.02.01 - Aplicações Financeiras Avaliadas a Valor Justo',
 '1.01.02.01.01 - Títulos para Negociação',
 '1.01.02.01.02 - Títulos Disponíveis para Venda',
 '1.01.02.02 - Aplicações Financeiras Avaliadas ao Custo Amortizado',
 '1.01.02.02.01 - Títulos Mantidos até o Vencimento',
 '1.01.03 - Contas a Receber',
 '1.01.03.01 - Clientes',
 '1.01.03.02 - Outras Contas a Receber',
 '1.01.04 - Estoques',
 '1.01.05 - Ativos Biológicos',
 '1.01.06 - Tributos a Recuperar',
 '1.01.06.01 - Tributos Correntes a Recuperar',
 '1.01.07 - Despesas Antecipadas',
 '1.01.08 - Outros Ativos Circulantes',
 '1.01.08.01 - Ativos Não-Correntes a Venda',
 '1.01.08.02 - Ativos de Operações Descontinuadas',
 '1.01.08.03 - Outros',
 '1.02 - Ativo Não Circulante',
 '1.02.01 - Ativo Realizável a Longo Prazo',
 '1.02.01.01 - Aplicações Financeiras Avaliadas a Valor Justo',
 '1.02.01.01.0

In [312]:
bpa_completo = pd.pivot_table(bpa_completo, index=['DENOM_CIA','DS_CONTA', 'CD_CONTA'], columns=['DT_FIM_EXERC'],values=['VL_AJUSTADO'])
bpa_completo    

VL_AJUSTADO  \
DT_FIM_EXERC                                                                                             2019-03-31   
DENOM_CIA                              DS_CONTA                                           CD_CONTA                    
ALIANSCE SONAE SHOPPING CENTERS S.A.   Aplicações Financeiras                             1.01.02               0.0   
                                       Aplicações Financeiras Avaliadas a Valor Justo     1.01.02.01            0.0   
                                                                                          1.02.01.01            0.0   
                                       Aplicações Financeiras Avaliadas a Valor Justo ... 1.01.02.02            NaN   
                                                                                          1.02.01.02            NaN   
...                                                                                                             ...   
MULTIPLAN - EMPREEND IMOBILIARIOS S.A. Títulos Disponíveis para Venda                     1.02.01.01.02         0.0   
                                       Títulos Mantidos até o Vencimento                  1.01.02.02.01         0.0   
                                                                                          1.02.01.02.01         0.0   
                                       Títulos para Negociação                            1.01.02.01.01    961123.0   
                                                                                          1.02.01.01.01         0.0   

                                                                                                                    \
DT_FIM_EXERC                                                                                            2019-06-30   
DENOM_CIA                              DS_CONTA                                           CD_CONTA                   
ALIANSCE SONAE SHOPPING CENTERS S.A.   Aplicações Financeiras                             1.01.02              0.0   
                                       Aplicações Financeiras Avaliadas a Valor Justo     1.01.02.01           NaN   
                                                                                          1.02.01.01           NaN   
                                       Aplicações Financeiras Avaliadas a Valor Justo ... 1.01.02.02           0.0   
                                                                                          1.02.01.02           0.0   
...                                                                                                            ...   
MULTIPLAN - EMPREEND IMOBILIARIOS S.A. Títulos Disponíveis para Venda                     1.02.01.01.02        NaN   
                                       Títulos Mantidos até o Vencimento                  1.01.02.02.01        NaN   
                                                                                          1.02.01.02.01        NaN   
                                       Títulos para Negociação                            1.01.02.01.01   881860.0   
                                                                                          1.02.01.01.01        NaN   

                                                                                                                    \
DT_FIM_EXERC                                                                                            2019-09-30   
DENOM_CIA                              DS_CONTA                                           CD_CONTA                   
ALIANSCE SONAE SHOPPING CENTERS S.A.   Aplicações Financeiras                             1.01.02         666726.0   
                                       Aplicações Financeiras Avaliadas a Valor Justo     1.01.02.01           NaN   
                                                                                          1.02.01.01           NaN   
                                       Aplicações Financeiras Avaliadas a Va

###4.3.2. Balanço Patrimonial Passivo (BPP)

O passivo, por outro lado, é uma obrigação, como dívidas e contas a pagar. Enquanto o ativo é um recurso disponível para uso, o passivo é um elemento que representará uma saída de recursos e consequente diminuição do ativo.

Assim como no caso anterior, o balanço patrimonial passivo divide as obrigações entre **passivo circulante** — cujo vencimento ocorre em até 12 meses — e **não circulante** — que pode ser pago em um prazo maior de 12 meses.

Os ativos e passivos circulantes e não circulantes são divididos em outras subcategorias, que apresentaremos mais adiante.

É muito importante acrescentar que as empresas listadas na bolsa acrescentam ao balanço patrimonial passivo o patrimônio líquido, que é o que sobra ao diminuir o passivo do ativo da empresa.

In [313]:
bpp_completo = pd.DataFrame()

for arquivo_xlsx in arquivos_xlsx:
  bpp = pd.read_excel(arquivo_xlsx, sheet_name='BPP')
  bpp_completo = bpp_completo.append(bpp)

In [314]:
lista_de_contas = pd.DataFrame()
lista_de_contas["CONTA"] = bpp_completo['CD_CONTA'] + " - " + bpp_completo['DS_CONTA']
lista_de_contas["CONTA"].unique().tolist()

['2 - Passivo Total',
 '2.01 - Passivo Circulante',
 '2.01.01 - Obrigações Sociais e Trabalhistas',
 '2.01.01.01 - Obrigações Sociais',
 '2.01.01.01.01 - Pessoal, Encargos, Beneficios Sociais e Prêmios',
 '2.01.01.02 - Obrigações Trabalhistas',
 '2.01.02 - Fornecedores',
 '2.01.02.01 - Fornecedores Nacionais',
 '2.01.02.02 - Fornecedores Estrangeiros',
 '2.01.03 - Obrigações Fiscais',
 '2.01.03.01 - Obrigações Fiscais Federais',
 '2.01.03.01.01 - Imposto de Renda e Contribuição Social a Pagar',
 '2.01.03.01.02 - Impostos a Recolher',
 '2.01.03.02 - Obrigações Fiscais Estaduais',
 '2.01.03.03 - Obrigações Fiscais Municipais',
 '2.01.04 - Empréstimos e Financiamentos',
 '2.01.04.01 - Empréstimos e Financiamentos',
 '2.01.04.01.01 - Em Moeda Nacional',
 '2.01.04.01.02 - Em Moeda Estrangeira',
 '2.01.04.02 - Debêntures',
 '2.01.04.03 - Financiamento por Arrendamento Financeiro',
 '2.01.05 - Outras Obrigações',
 '2.01.05.01 - Passivos com Partes Relacionadas',
 '2.01.05.01.01 - Débitos com 

In [315]:
bpp_completo = pd.pivot_table(bpp_completo, index=['DENOM_CIA','DS_CONTA', 'CD_CONTA'], columns=['DT_FIM_EXERC'], values=['VL_AJUSTADO'])
bpp_completo

VL_AJUSTADO  \
DT_FIM_EXERC                                                                                      2019-03-31   
DENOM_CIA                              DS_CONTA                                    CD_CONTA                    
ALIANSCE SONAE SHOPPING CENTERS S.A.   (-)Gastos com emissões de ações             2.03.02.07            NaN   
                                       Adiantamento para Futuro Aumento de Capital 2.02.02.02.02         0.0   
                                                                                   2.03.02.06            0.0   
                                       Ajustes Acumulados de Conversão             2.03.07               0.0   
                                       Ajustes de Avaliação Patrimonial            2.03.06               0.0   
...                                                                                                      ...   
MULTIPLAN - EMPREEND IMOBILIARIOS S.A. Subvenções de Investimento a Apropriar      2.02.06.03            0.0   
                                       Tributos Diferidos                          2.02.03          212148.0   
                                       Valores a pagar a partes relacionadas       2.01.05.02.09         NaN   
                                                                                   2.02.02.02.09         NaN   
                                       Ágio na Emissão de Ações                    2.03.02.01       706538.0   

                                                                                                             \
DT_FIM_EXERC                                                                                     2019-06-30   
DENOM_CIA                              DS_CONTA                                    CD_CONTA                   
ALIANSCE SONAE SHOPPING CENTERS S.A.   (-)Gastos com emissões de ações             2.03.02.07           NaN   
                                       Adiantamento para Futuro Aumento de Capital 2.02.02.02.02        0.0   
                                                                                   2.03.02.06           0.0   
                                       Ajustes Acumulados de Conversão             2.03.07              0.0   
                                       Ajustes de Avaliação Patrimonial            2.03.06              0.0   
...                                                                                                     ...   
MULTIPLAN - EMPREEND IMOBILIARIOS S.A. Subvenções de Investimento a Apropriar      2.02.06.03           0.0   
                                       Tributos Diferidos                          2.02.03         214146.0   
                                       Valores a pagar a partes relacionadas       2.01.05.02.09        NaN   
                                                                                   2.02.02.02.09        NaN   
                                       Ágio na Emissão de Ações                    2.03.02.01      707347.0   

                                                                                                             \
DT_FIM_EXERC                                                                                     2019-09-30   
DENOM_CIA                              DS_CONTA                                    CD_CONTA                   
ALIANSCE SONAE SHOPPING CENTERS S.A.   (-)Gastos com emissões de ações             2.03.02.07      -16084.0   
                                       Adiantamento para Futuro Aumento de Capital 2.02.02.02.02        0.0   
                                                                                   2.03.02.06           0.0   
                                       Ajustes Acumulados de Conversão             2.03.07              0.0   
                                       Ajustes de Avaliação Patrimonial            2.03.06        1480677.0   
...                                                                                  

#5. Indicadores Fundamentalistas


##5.1. Funções para plotar os gráficos (anual e trimestral)

In [402]:
def gera_grafico_anual(df_anual, titulo, descricao_eixo_y, unidade_eixo_y, unidade_y_apos=True):
    fig = go.Figure()

    for c in df_anual.columns:

        if unidade_y_apos:
            template_y = "%{value:.2f} " + unidade_eixo_y
            hover_template_y = "<b>" + titulo + "</b><br>Data: %{label}<br>" + descricao_eixo_y + ": %{y:.2f} " + unidade_eixo_y
        else:
            template_y = unidade_eixo_y + " %{value:.2f}"
            hover_template_y = "<b>" + titulo + "</b><br>Data: %{label}<br>" + descricao_eixo_y + ": " + unidade_eixo_y + " %{y:.2f} "

        fig.add_trace(go.Bar(x=df_anual.index,
                             y=df_anual[c],
                             name=c, text=df_anual[c], 
                             texttemplate = template_y,
                             hovertemplate = hover_template_y,
                             textposition="inside",
                             textangle=0,
                             textfont={'family': "Arial", 'size': 15, 'color': "Black"},
                             ))
    fig.update_layout(
        yaxis_title=descricao_eixo_y + " (" + unidade_eixo_y + ")",
        xaxis_title="Anos",
        legend_title="Empresas selecionadas:",    
        title={'text': '<b>' + titulo + ' Anual para os anos de ' + str(anos),
              'y':0.9, 'x':0.5,
              'xanchor': 'center', 'yanchor': 'top'},
        legend=dict(x=0, y=-0.5),
        barmode='group',
        bargap=0.2, # gap between bars of adjacent location coordinates.
        bargroupgap=0.1) # gap between bars of the same location coordinate.

    # Pega a data em que o gráfico foi gerado
    today = date.today().strftime('%d/%m/%Y')

    fig.add_annotation(x=0.9, y=0, text=f"Fonte dos dados: https://dados.cvm.gov.br/ <br>Data da geração: {today}", showarrow=False,
                      xref='paper', yref='paper', 
                      xshift=150, yshift=-130
        , font=dict(size=12, color="grey")
        , align="left"
        )

    fig.show()

def gera_grafico_trimestral(df_trimestral, titulo):
    fig = go.Figure()

    for c in df_trimestral.columns:
        fig.add_trace(go.Bar(x=df_trimestral.index,
                            y=df_trimestral[c],
                            name=c, 
                            hovertemplate = "<b>" + titulo + "</b><br>Data: %{label}<br>Percentual: %{y:.2f} %",
                            text=df_trimestral[c], 
                            texttemplate="%{value:.2f} %",
                            textposition="inside",
                            textangle=0,
                            textfont={'family': "Arial", 'size': 15, 'color': "Black"},
                            ))
    fig.update_xaxes(tickangle=-80, )

    fig.update_layout(
        #autosize=False,
        #width=1200,
        #height=800,
        yaxis_title="Percentual (%)",
        xaxis_title="Trimestres/Anos",
        legend_title="Empresas selecionadas:",
        title={
            'text': '<b>' + titulo + ' Trimestral para os anos de ' + str(anos),
            'y':0.9,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'},
        legend=dict(x=0, y=-0.8),
        barmode='group',
        bargap=0.2, # gap between bars of adjacent location coordinates.
        bargroupgap=0.1) # gap between bars of the same location coordinate.

    # Pega a data em que o gráfico foi gerado

    # fig.update_xaxes(
    #     showgrid=True,
    #     ticks="outside",
    #     tickson="boundaries",
    #     ticklen=20
    # )

    today = date.today().strftime('%d/%m/%Y')
    fig.add_annotation(x=0.9, y=0, text=f"Fonte dos dados: https://dados.cvm.gov.br/ <br>Data da geração: {today}", showarrow=False,
                      xref='paper', yref='paper', xshift=150, yshift=-130, font=dict(size=12, color="grey"), align="left")

    #fig.update_layout(hovermode="x unified")
    # fig.update_layout(
    #     hoverlabel=dict(
    #         bgcolor="white",
    #         font_size=16,
    #         font_family="Rockwell"
    #     )
    # )
    fig.show()

##5.2. Margens

### 5.2.1. Margem Bruta (usa a DRE)

#### 5.2.1.1. Descrição

***Referência: https://statusinvest.com.br/termos/m/margem-bruta***

A Margem Bruta é uma métrica utilizada para a identificação da rentabilidade bruta de uma empresa, sendo a razão entre o Lucro Bruto e a Receita Líquida. Assim, através da margem bruta é possível tirar algumas conclusões sobre a eficiência da empresa. Isso ocorre, pois o investidor consegue calcular o quanto a empresa tem de lucro pelos seus investimentos, após a dedução de:

- Impostos sobre Vendas;
- Devoluções;
- Descontos;
- Custos de material;
- Custo de Produto Vendido (CPV).

Para calcular Margem Bruta, basta utilizar a seguinte fórmula:

$MargemBruta = \frac{LucroBruto}{ReceitaLiquida} \times 100$

Os números referentes ao Lucro Bruto e Receita Líquida podem ser recolhidos nos relatórios financeiros oferecidos aos investidores pelas empresas de capital aberto na bolsa. A margem bruta é bastante importante para gestores e investidores, já que produz informações relevantes sobre a rentabilidade da empresa.

#### 5.1.1.2. Ajustando os DataFrames

In [317]:
margem_bruta_setor = pd.DataFrame()

for i in range(0, len(lista_de_empresas)):
    resultado_bruto = dre_completo.loc[lista_de_empresas[i],:].loc['Resultado Bruto']
    receita = dre_completo.loc[lista_de_empresas[i],:].loc['Receita de Venda de Bens e/ou Serviços']
    margem_bruta = pd.Series(resultado_bruto / receita) * 100.0
    margem_bruta_setor = pd.concat([margem_bruta_setor, margem_bruta], axis=1)

margem_bruta_setor.columns = lista_de_empresas

In [318]:
# Cria um df anual
margem_bruta_setor_anual = margem_bruta_setor.copy(deep=True)
margem_bruta_setor_anual = margem_bruta_setor_anual[margem_bruta_setor_anual.index.get_level_values(1).str[4:8] =='-01-']
margem_bruta_setor_anual = margem_bruta_setor_anual[margem_bruta_setor_anual.index.get_level_values(2).str[4:8] =='-12-']
margem_bruta_setor_anual.index = margem_bruta_setor_anual.index.get_level_values(1).str[:4]

anos = list(margem_bruta_setor_anual.index.unique()) # pega os anos, normalmente tem um ano a mais que os anos de interesse
                                                     # se não quiser pode filtrar só pelo ULTIMO valor

In [319]:
# Cria um df trimestral
def retorna_trimestre(df):
    if df['MES_INICIAL'] == '01' and df['MES_FINAL'] == '03':
        return str(df['ANO']) + '-01-01' 
    if df['MES_INICIAL'] == '04' and df['MES_FINAL'] == '06':
        return str(df['ANO']) + '-04-01' 
    if df['MES_INICIAL'] == '07' and df['MES_FINAL'] == '09':
        return str(df['ANO']) + '-07-01' 
    if df['MES_INICIAL'] == '10' and df['MES_FINAL'] == '12':
        return str(df['ANO']) + '-10-01' 
    else:
        return None

# Cria um df por trimestre
margem_bruta_setor_trimestral = margem_bruta_setor.copy(deep=True)

margem_bruta_setor_trimestral['MES_INICIAL'] = margem_bruta_setor_trimestral.index.get_level_values(1).str[5:7]
margem_bruta_setor_trimestral['MES_FINAL'] = margem_bruta_setor_trimestral.index.get_level_values(2).str[5:7]
margem_bruta_setor_trimestral['ANO'] = margem_bruta_setor_trimestral.index.get_level_values(1).str[:4]
margem_bruta_setor_trimestral['TRIMESTRE'] = margem_bruta_setor_trimestral.apply(retorna_trimestre, axis=1)

margem_bruta_setor_trimestral.index = margem_bruta_setor_trimestral['TRIMESTRE']
margem_bruta_setor_trimestral[margem_bruta_setor_trimestral['TRIMESTRE'].str[-6:]=='-10-01']

anos = list(margem_bruta_setor_trimestral['ANO'].unique()) # pega os anos, normalmente tem um ano a mais que os anos de interesse
                                                           # se não quiser pode filtrar só pelo ULTIMO valor

margem_bruta_setor_trimestral = margem_bruta_setor_trimestral.drop('TRIMESTRE', axis=1)
margem_bruta_setor_trimestral = margem_bruta_setor_trimestral.drop('MES_INICIAL', axis=1)
margem_bruta_setor_trimestral = margem_bruta_setor_trimestral.drop('MES_FINAL', axis=1)
margem_bruta_setor_trimestral = margem_bruta_setor_trimestral.drop('ANO', axis=1)

margem_bruta_setor_trimestral.sort_index(ascending=False, inplace=True)
margem_bruta_setor_trimestral = margem_bruta_setor_trimestral[margem_bruta_setor_trimestral.index.notnull()]
#margem_bruta_setor_trimestral

#### 5.1.1.3. Cria o gráfico anual

In [403]:
gera_grafico_anual(margem_bruta_setor_anual, "Margem Bruta", "Percentual", "%")

#### 5.1.1.4. Cria o gráfico trimestral

In [321]:
gera_grafico_trimestral(margem_bruta_setor_trimestral, "Margem Bruta")

### 5.2.2. Margem Líquida (usa a DRE)

#### 5.2.2.1. Descrição

***Referência: https://statusinvest.com.br/termos/m/margem-liquida***

A Margem Líquida é razão entre o Lucro Líquido e a Receita Líquida de uma companhia após a dedução de impostos e tributos. Podendo representar um resultado trimestral ou anual, a Margem Líquida representa o resultado líquido das vendas de um negócio. Portanto, está diretamente ligado com o nível de rentabilidade que uma companhia consegue com suas operações. Para os investidores, a Margem Líquida demonstra se uma empresa possui bons retornos a partir custos de produção do seu produto/serviço.

A Margem do Lucro Liquido possibilita aos investidores uma boa avaliação sobre a realidade financeira de uma companhia. Em caso de margem negativa, é possível concluir que a empresa em questão possui prejuízo, já que tanto a receita quanto o lucro depois dos impostos, despesas e custos são levados em conta.

No entanto, é importante lembrar que nem sempre prejuízos significam algo negativo, já que existem algumas variáveis que podem causar esse resultado.

No caso de uma margem alta, a indicação é de que a companhia possui uma boa vantagem competitiva, indicando que seus projetos devem estar dando resultados positivos e é vantajoso o investimento nela. Esse resultado pode ser causado por uma boa exposição da marca ou pelo lançamento de um novo produto ou serviço no mercado, por exemplo.


#### 5.2.2.2. Ajustando os DataFrames

In [322]:
margem_liquida_setor = pd.DataFrame()

for i in range(0, len(lista_de_empresas)):
    lucro_liquido = dre_completo.loc[lista_de_empresas[i],:].loc['Lucro/Prejuízo Consolidado do Período']
    receita_liquida = dre_completo.loc[lista_de_empresas[i],:].loc['Receita de Venda de Bens e/ou Serviços']
    margem_liquida = pd.Series(lucro_liquido / receita_liquida) * 100.0
    margem_liquida_setor = pd.concat([margem_liquida_setor, margem_liquida], axis=1)

margem_liquida_setor.columns = lista_de_empresas

In [323]:
# Cria um df anual
margem_liquida_setor_anual = margem_liquida_setor.copy(deep=True)
margem_liquida_setor_anual = margem_liquida_setor_anual[margem_liquida_setor_anual.index.get_level_values(1).str[4:8] =='-01-']
margem_liquida_setor_anual = margem_liquida_setor_anual[margem_liquida_setor_anual.index.get_level_values(2).str[4:8] =='-12-']
margem_liquida_setor_anual.index = margem_liquida_setor_anual.index.get_level_values(1).str[:4]

anos = list(margem_liquida_setor_anual.index.unique()) # pega os anos, normalmente tem um ano a mais que os anos de interesse
                                                       # se não quiser pode filtrar só pelo ULTIMO valor

In [324]:
# Cria um df por trimestre
margem_liquida_setor_trimestral = margem_liquida_setor.copy(deep=True)

margem_liquida_setor_trimestral['MES_INICIAL'] = margem_liquida_setor_trimestral.index.get_level_values(1).str[5:7]
margem_liquida_setor_trimestral['MES_FINAL'] = margem_liquida_setor_trimestral.index.get_level_values(2).str[5:7]
margem_liquida_setor_trimestral['ANO'] = margem_liquida_setor_trimestral.index.get_level_values(1).str[:4]
margem_liquida_setor_trimestral['TRIMESTRE'] = margem_liquida_setor_trimestral.apply(retorna_trimestre, axis=1)
margem_liquida_setor_trimestral.index = margem_liquida_setor_trimestral['TRIMESTRE']
margem_liquida_setor_trimestral[margem_liquida_setor_trimestral['TRIMESTRE'].str[-6:]=='-10-01']

anos = list(margem_liquida_setor_trimestral['ANO'].unique()) # pega os anos, normalmente tem um ano a mais que os anos de interesse
                                                             # se não quiser pode filtrar só pelo ULTIMO valor

margem_liquida_setor_trimestral = margem_liquida_setor_trimestral.drop('TRIMESTRE', axis=1)
margem_liquida_setor_trimestral = margem_liquida_setor_trimestral.drop('MES_INICIAL', axis=1)
margem_liquida_setor_trimestral = margem_liquida_setor_trimestral.drop('MES_FINAL', axis=1)
margem_liquida_setor_trimestral = margem_liquida_setor_trimestral.drop('ANO', axis=1)

margem_liquida_setor_trimestral.sort_index(ascending=False, inplace=True)
margem_liquida_setor_trimestral = margem_liquida_setor_trimestral[margem_liquida_setor_trimestral.index.notnull()]
#margem_bruta_setor_trimestral

#### 5.2.2.3. Cria o gráfico anual

In [404]:
gera_grafico_anual(margem_liquida_setor_anual, "Margem Líquida", "Percentual", "%")

#### 5.2.2.4. Cria o gráfico trimestral

In [326]:
gera_grafico_trimestral(margem_liquida_setor_trimestral, "Margem Líquida")

##5.3. Dívida Bruta/PL (Patrimônio Líquido) (usa o Balanço Patrimonial Passivo - BPP)

###5.3.1. Descrição

***Referência: https://statusinvest.com.br/termos/d/divida-bruta-patrimonio-liquido*** 

Em suma, a **Dívida Bruta/PL** faz um comparativo sobre as questões que envolvem a área de endividamento e alavancagem da empresa, com a sua riqueza total, isto é a diferença entre os passivos e ativos que o negócio possui.

Dessa forma, este se torna um indicador importante no momento de se avaliar a saúde financeira do negócio, pois é possível compreender a capacidade de endividamento que um negócio pode ter, sem comprometer seu valor real.

Afinal, o indicador de **Dívida Bruta/PL** avalia o total de dívidas que o negócio possui sobre o valor real que a empresa tem. Portanto, para o investidor, saber interpretar esse dado é útil no momento de avaliar um investimento, pois tal informação aponta para real saúde financeira da empresa e sua capacidade alavancagem, ou seja, investidores de longo prazo conseguem ter um cenário aproximado sobre o potencial que a empresa tem e o risco que ela pode representar.

### 5.3.2. Ajustando os DataFrames

In [327]:
# Fazer um ajuste no patrimônio líquido para remover a conta 'Participação dos Acionistas Não Controladores'
pl_ajustado_setor = pd.DataFrame()

for i in range(0,len(lista_de_empresas)):
    pl_consolidado = bpp_completo.loc[lista_de_empresas[i],:].loc['Patrimônio Líquido Consolidado'].loc["2.03"]
    participacao_nao_controladores = bpp_completo.loc[lista_de_empresas[i],:].loc['Participação dos Acionistas Não Controladores'].loc["2.03.09"]
    pl_ajustado = pd.Series(pl_consolidado - participacao_nao_controladores)     
    pl_ajustado_setor = pd.concat([pl_ajustado_setor, pl_ajustado], axis=1)

pl_ajustado_setor.columns = lista_de_empresas
# pl_ajustado_setor

In [328]:
# 2.01	Passivo Circulante
# 2.01.04	Empréstimos e Financiamentos
# 2.01.04.01	Empréstimos e Financiamentos

# 2.02	Passivo Não Circulante (prazo de vencimento superior a um ano)	
# 2.02.01	Empréstimos e Financiamentos
# 2.02.01.01	Empréstimos e Financiamentos

divida_bruta_sobre_pl_setor = pd.DataFrame()

for i in range(0, len(lista_de_empresas)):
    divida_curto_prazo = bpp_completo.loc[lista_de_empresas[i],:].loc['Empréstimos e Financiamentos'].loc["2.01.04"]
    divida_longo_prazo = bpp_completo.loc[lista_de_empresas[i],:].loc['Empréstimos e Financiamentos'].loc["2.02.01"]
    divida_bruta = pd.Series(divida_curto_prazo + divida_longo_prazo)
    divida_bruta_sobre_pl_setor = pd.concat([divida_bruta_sobre_pl_setor, divida_bruta], axis=1)
    
divida_bruta_sobre_pl_setor.columns = lista_de_empresas
divida_bruta_sobre_pl_setor = (divida_bruta_sobre_pl_setor/pl_ajustado_setor) * 100.0
# divida_bruta_sobre_pl_setor

In [330]:
# Cria um df anual
divida_bruta_sobre_pl_setor_anual = divida_bruta_sobre_pl_setor.copy(deep=True)
divida_bruta_sobre_pl_setor_anual = divida_bruta_sobre_pl_setor_anual[divida_bruta_sobre_pl_setor_anual.index.get_level_values(1).str[4:8] =='-12-']
divida_bruta_sobre_pl_setor_anual.index = divida_bruta_sobre_pl_setor_anual.index.get_level_values(1).str[:4]

anos = list(divida_bruta_sobre_pl_setor_anual.index.unique()) # pega os anos, normalmente tem um ano a mais que os anos de interesse
                                                              # se não quiser pode filtrar só pelo ULTIMO valor

###5.3.3. Cria o gráfico anual

In [405]:
gera_grafico_anual(divida_bruta_sobre_pl_setor_anual, "Dívida Bruta/PL", "Percentual", "%")

###5.3.4. Cria o gráfico trimestral

In [333]:
# TODO

##5.4. Disponibilidades


###5.4.1. Disponibilidades


####5.4.1.1. Descrição

TODO

#### 5.4.1.2. Ajustando os DataFrames

In [365]:
caixa_ajustado_setor = pd.DataFrame()

for i in range(0, len(lista_de_empresas)):
    caixa_ajustado = pd.Series(bpa_completo.loc[lista_de_empresas[i],:].loc['Caixa e Equivalentes de Caixa'].loc['1.01.01'] +
                               bpa_completo.loc[lista_de_empresas[i],:].loc['Aplicações Financeiras'].loc['1.01.02'])     
    caixa_ajustado_setor = pd.concat([caixa_ajustado_setor, caixa_ajustado], axis=1)

caixa_ajustado_setor.columns = lista_de_empresas

In [394]:
# Cria um df anual
caixa_ajustado_setor_anual = caixa_ajustado_setor.copy(deep=True)
caixa_ajustado_setor_anual = caixa_ajustado_setor_anual[caixa_ajustado_setor_anual.index.get_level_values(1).str[4:8] =='-12-']
caixa_ajustado_setor_anual.index = caixa_ajustado_setor_anual.index.get_level_values(1).str[:4]

anos = list(caixa_ajustado_setor_anual.index.unique()) # pega os anos, normalmente tem um ano a mais que os anos de interesse
                                                       # se não quiser pode filtrar só pelo ULTIMO valor

caixa_ajustado_setor_anual                                                       

,ALIANSCE SONAE SHOPPING CENTERS S.A.,BR MALLS PARTICIPACOES S.A.,MULTIPLAN - EMPREEND IMOBILIARIOS S.A.,IGUATEMI EMPRESA DE SHOPPING CENTERS S.A
2019,1200048.0,781224.0,907969.0,969584.0
2020,1384700.0,1445811.0,1223414.0,1571202.0
2021,1246322.0,1397845.0,778463.0,1790137.0
2022,NaN,NaN,1024991.0,1078680.0


####5.4.1.3. Cria o gráfico anual

In [406]:
gera_grafico_anual(caixa_ajustado_setor_anual, 'Disponibilidades', "Valor", "R$", False)

####5.4.1.4. Cria o gráfico trimestral

In [368]:
# TODO

###5.3.2. Liquidez Corrente

####5.3.2.1. Descrição

***Referência: https://statusinvest.com.br/termos/l/liquidez-corrente***

A liquidez corrente é um indicador usado para mensurar, a curto prazo de tempo, a capacidade de pagamento de uma companhia. Desse modo, a liquidez corrente busca obter a razão entre o ativo circulante da companhia pelo passivo circulante.

Dessa forma, através da liquidez corrente é possível analisar a capacidade de pagamento de uma empresa, que pode ajudar a identificar a saúde do negócio.

A liquidez corrente possui o propósito de mensurar a capacidade de pagamento de uma companhia. Desse modo, existem algumas interpretações da liquidez corrente que possibilitam a compreensão dos resultados, dentre essas interpretações, estão:

- **Maior que 1** – Caso a liquidez corrente apresente um resultado maior que 1, significa que a companhia possui uma capacidade de pagamento relativamente boa, ou seja, expõe que a empresa possui um líquido positivo;

- **Menor que 1** – Entretanto, se o indicativo da liquidez corrente apresentar um resultado inferior a 1,  demonstra que a empresa como dependente de geração de caixa, dado que, o ativo circulante está inferior ao passivo circulante;

- **Igual a 1** – Além disso, caso o indicativo da liquidez corrente apresente um resultado igual a 1, revela que o capital da empresa e seus pagamentos estão proporcionais.


In [369]:
liquidez_corrente_empresas = pd.DataFrame()

for i in range(0, len(lista_de_empresas)):
    ativo_circulante = bpa_completo.loc[lista_de_empresas[i],:].loc['Ativo Circulante'].loc['1.01']
    passivo_circulante = bpp_completo.loc[lista_de_empresas[i],:].loc['Passivo Circulante'].loc['2.01']
    liquidez_corrente = pd.Series(ativo_circulante/passivo_circulante)     
    liquidez_corrente_empresas = pd.concat([liquidez_corrente_empresas, liquidez_corrente], axis=1)

liquidez_corrente_empresas.columns = lista_de_empresas
# liquidez_corrente_empresas

##5.5. EBIT

### 5.5.1. Descrição

***Referência: https://statusinvest.com.br/termos/e/ebit***

O **EBIT**, ou *Earnings Before Interest and Taxes*, é um indicador que demonstra o lucro da empresa antes do reconhecimento de suas despesas com juros e impostos.

Em português, o **EBIT** é chamado de LAJIR, ou Lucro Antes de Juros e Impostos. Através dele, é possível entender melhor os resultados operacionais informados no Demonstrativo de Resultados de exercício (DRE) de uma companhia.

Para fazer o cálculo do EBIT é necessário lembrar que esse indicador representa o lucro da empresa antes do pagamento de juros e impostos. Portanto, basta somar os encargos líquidos, impostos sobre lucros e encargos financeiros de financiamento, a partir da seguinte fórmula:

$EBIT=LucroLiquido + ResultadoFinanceiro + Impostos$

Para encontrar essas informações, basta consultar o Demonstrativo de Resultados do Exercício (DRE) da empresa em questão. Esse documento contábil é divulgado trimestralmente pelas companhias de capital aberto na bolsa de valores.

In [370]:
ebit_ajustado_empresas = pd.DataFrame()

for i in range(0, len(lista_de_empresas)):
    ebit_ajustado = pd.Series(dre_completo.loc[lista_de_empresas[i],:].loc['Receita de Venda de Bens e/ou Serviços'] +
                              dre_completo.loc[lista_de_empresas[i],:].loc['Custo dos Bens e/ou Serviços Vendidos'] + 
                              dre_completo.loc[lista_de_empresas[i],:].loc['Despesas com Vendas'] +
                              dre_completo.loc[lista_de_empresas[i],:].loc['Despesas Gerais e Administrativas'])
    ebit_ajustado_empresas = pd.concat([ebit_ajustado_empresas, ebit_ajustado], axis=1)

ebit_ajustado_empresas.columns = lista_de_empresas

# fig = go.Figure(go.Bar(x=ebit_ajustado.iloc[-1],y=tickers,orientation='h'))
# fig.update_layout(title_text='Resultado operacional EBIT')
# fig.show()

In [371]:
# Cria um df anual
ebit_ajustado_empresas_anual = ebit_ajustado_empresas.copy(deep=True)
ebit_ajustado_empresas_anual = ebit_ajustado_empresas_anual[ebit_ajustado_empresas_anual.index.get_level_values(1).str[4:8] =='-01-']
ebit_ajustado_empresas_anual = ebit_ajustado_empresas_anual[ebit_ajustado_empresas_anual.index.get_level_values(2).str[4:8] =='-12-']
ebit_ajustado_empresas_anual.index = ebit_ajustado_empresas_anual.index.get_level_values(1).str[:4]

anos = list(ebit_ajustado_empresas_anual.index.unique()) # pega os anos, normalmente tem um ano a mais que os anos de interesse
                                                       # se não quiser pode filtrar só pelo ULTIMO valor

ebit_ajustado_empresas_anual                                                       

,ALIANSCE SONAE SHOPPING CENTERS S.A.,BR MALLS PARTICIPACOES S.A.,MULTIPLAN - EMPREEND IMOBILIARIOS S.A.,IGUATEMI EMPRESA DE SHOPPING CENTERS S.A
2019,415981.0,912564.0,781339.0,406075.0
2020,340741.0,531288.0,569088.0,316414.0
2021,441707.0,680499.0,723904.0,378642.0
2022,NaN,NaN,1152947.0,446138.0


In [407]:
gera_grafico_anual(ebit_ajustado_empresas_anual, "EBIT", "Valor", "R$", False)

In [ ]:
#Margem operacional

margem_operacional = pd.DataFrame()

for i in range(0,n_empresas):
  margem_operacional_ = pd.Series((dre.loc[lista_de_empresas[i],:].loc['Resultado Bruto'].iloc[-1])/(dre.loc[lista_de_empresas[i],:].loc['Receita de Venda de Bens e/ou Serviços'].iloc[-1,]+
                             dre.loc[lista_de_empresas[i],:].loc['Custo dos Bens e/ou Serviços Vendidos'].iloc[-1,]+ 
                             dre.loc[lista_de_empresas[i],:].loc['Despesas com Vendas'].iloc[-1,]+
                             dre.loc[lista_de_empresas[i],:].loc['Despesas Gerais e Administrativas'].iloc[-1,]))
  margem_operacional = pd.concat([margem_operacional,margem_operacional_], axis=1)
margem_operacional.columns = lista_de_empresas
fig = go.Figure(go.Bar(x=margem_operacional.iloc[-1],y=tickers,orientation='h'))
fig.update_layout(title_text='Margem Operacional')
fig.show()

##5.4. Ebitda/Div Bruta

In [ ]:
#Ebitda/Divida Bruta

ebit_div_bruta = pd.DataFrame()

for i in range(0,n_empresas):
  ebit_div_ajustado_ = pd.Series((dre.loc[lista_de_empresas[i],:].loc['Receita de Venda de Bens e/ou Serviços'].iloc[-1,]+
                             dre.loc[lista_de_empresas[i],:].loc['Custo dos Bens e/ou Serviços Vendidos'].iloc[-1,]+ 
                             dre.loc[lista_de_empresas[i],:].loc['Despesas com Vendas'].iloc[-1,]+
                             dre.loc[lista_de_empresas[i],:].loc['Despesas Gerais e Administrativas'].iloc[-1,])/
                             (bpp.loc[lista_de_empresas[i],:].loc['Empréstimos e Financiamentos'].iloc[-2,3] + bpp.loc[lista_de_empresas[i],:].loc['Empréstimos e Financiamentos'].iloc[0,3]))
  ebit_div_bruta = pd.concat([ebit_div_bruta, ebit_div_ajustado_], axis=1)
ebit_div_bruta.columns = lista_de_empresas

In [ ]:
ebit_div_bruta

In [ ]:
fig = go.Figure(go.Bar(
    x=ebit_div_bruta.iloc[-1],
    y=tickers,
    orientation='h'
))
fig.update_layout(title_text='EBIT/Divida Bruta')
fig.show()

##5.5. Return on Equity - ROE



In [ ]:
#ROE E ROIC 
demonstrativos = ['DFC_MI','BPA', 'DRE', 'BPP']
lista_listas = []
a = 0

link = 'http://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/ITR/DADOS/itr_cia_aberta_2022.zip'
r = requests.get(link)

for j in empresas:
  c = 0
  lista_df = []

  for k in demonstrativos:
    zf = zipfile.ZipFile(io.BytesIO(r.content))
    arquivo = 'itr_cia_aberta_' + str(k) + '_con_2022.csv'
    zf = zf.open(arquivo)
    lines = zf.readlines()
    lines = [i.strip().decode('ISO-8859-1') for i in lines]
    lines = [i.split(';') for i in lines]
    c += 1 
    df = pd.DataFrame(lines[1:], columns = lines[0])
    df['VL_AJUSTADO'] = pd.to_numeric(df['VL_CONTA'])
    filtro = df[df['CD_CVM']== '0' + str(j)]
    lista_df.append(filtro)
    print(f'Trabalhando com a empresa {j} e sua demonstração {k}. As dimensões são {filtro.shape} ')

  lista_listas.append(lista_df)
  writer = pd.ExcelWriter(f'Demonstrativos Empresa {str(j)}.xlsx', engine='xlsxwriter')
  lista_listas[a][0].to_excel(writer, sheet_name='DFC_MD',encoding= 'ISO-8859-1')
  lista_listas[a][1].to_excel(writer, sheet_name='DFC_MI',encoding= 'ISO-8859-1')
  lista_listas[a][2].to_excel(writer, sheet_name='BPA',encoding= 'ISO-8859-1')
  lista_listas[a][3].to_excel(writer, sheet_name='DRE',encoding= 'ISO-8859-1')
  lista_listas[a][4].to_excel(writer, sheet_name='BPP',encoding= 'ISO-8859-1')
  a += 1
  print(f'Arquivo excel com os demonstrativos da empresa {j} já exportado. \n')
  writer.save()

print("O tempo de execução desse programa foi de %s segundos ---" % (time.time() - start_time))

In [ ]:
dre2 = pd.DataFrame()
for f in arquivos_xls:
  dre2_xls = pd.read_excel(f, sheet_name='DRE')
  dre2 = dre2.append(dre2_xls)

In [ ]:
dre2 = pd.pivot_table(dre2, index=['DENOM_CIA','DS_CONTA'], columns=['DT_INI_EXERC', 'DT_FIM_EXERC'],values=['VL_AJUSTADO'])

In [ ]:
dre2.loc['BR MALLS PARTICIPACOES S.A.',:].loc['Receita de Venda de Bens e/ou Serviços']

In [ ]:
# PEGANDO DADOS ANUAIS CVM, O REPOSITÓRIO É DEMONSTRAÇÕES FINANCEIRAS PADRONIZADAS

# Remover demais demonstrações,manter somente DRE
demonstrativos = ['DRE']
lista_listas = []
a = 0

# Modificar o link com o link direto do zip da cvm
link = 'http://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/DFP/DADOS/dfp_cia_aberta_2022.zip'
r = requests.get(link)

for j in empresas:
  c = 0
  lista_df = []
  for k in demonstrativos:
    zf = zipfile.ZipFile(io.BytesIO(r.content))
    arquivo = 'dfp_cia_aberta_' + str(k) + '_con_2022.csv'
    zf = zf.open(arquivo)
    lines = zf.readlines()
    lines = [i.strip().decode('ISO-8859-1') for i in lines]
    lines = [i.split(';') for i in lines]
    c += 1 
    df = pd.DataFrame(lines[1:], columns = lines[0])
    df['VL_AJUSTADO'] = pd.to_numeric(df['VL_CONTA'])
    filtro = df[df['CD_CVM']== '0' + str(j)]
    lista_df.append(filtro)

    print(f'Trabalhando com a empresa {j} e sua demonstração {k}. As dimensões são {filtro.shape} ')

  lista_listas.append(lista_df)

  # Manter
  writer = pd.ExcelWriter(f'Demonstrativos Empresa {str(j)}.xlsx', engine='xlsxwriter')
  # Deixar apenas um repositório
  lista_listas[a][0].to_excel(writer, sheet_name='DRE',encoding= 'ISO-8859-1')
  a += 1
  print(f'Arquivo excel com os demonstrativos da empresa {j} já exportado. \n')
  writer.save()

print("O tempo de execução desse programa foi de %s segundos ---" % (time.time() - start_time))


In [ ]:
dre3 = pd.DataFrame()
for f in arquivos_xls:
  dre3_xls = pd.read_excel(f, sheet_name='DRE')
  dre3 = dre3.append(dre3_xls)

In [ ]:
dre3 = pd.pivot_table(dre3, index=['DENOM_CIA','DS_CONTA'], columns=['DT_INI_EXERC', 'DT_FIM_EXERC'],values=['VL_AJUSTADO'])

In [ ]:
dre3

In [ ]:
dre.loc['BR MALLS PARTICIPACOES S.A.',:].loc['Atribuído a Sócios da Empresa Controladora'].iloc[-1]

In [ ]:
dre.loc['BR MALLS PARTICIPACOES S.A.',:].loc['Atribuído a Sócios da Empresa Controladora']

In [ ]:
dre2.loc['BR MALLS PARTICIPACOES S.A.',:].loc['Atribuído a Sócios da Empresa Controladora']

In [ ]:
dre3.loc['BR MALLS PARTICIPACOES S.A.',:].loc['Atribuído a Sócios da Empresa Controladora']

In [ ]:
ll_Q4_brmalls = dre3.loc['BR MALLS PARTICIPACOES S.A.',:].loc['Atribuído a Sócios da Empresa Controladora'].iloc[-1] - (
    dre2.loc['BR MALLS PARTICIPACOES S.A.',:].loc['Atribuído a Sócios da Empresa Controladora'].iloc[-5]+
    dre2.loc['BR MALLS PARTICIPACOES S.A.',:].loc['Atribuído a Sócios da Empresa Controladora'].iloc[-2]+
    dre2.loc['BR MALLS PARTICIPACOES S.A.',:].loc['Atribuído a Sócios da Empresa Controladora'].iloc[-1]
)


In [ ]:
ll_Q4_brmalls

In [ ]:
Q320 = pd.DataFrame()

for i in range(0,n_empresas):
  Q320_ = pd.Series(dre2.loc[lista_de_empresas[i],:].loc['Atribuído a Sócios da Empresa Controladora'].iloc[-1])
  Q320 = pd.concat([Q320,Q320_], axis=1)
Q320.columns = lista_de_empresas
Q320

In [ ]:
Q420 = pd.DataFrame()

for i in range(0,n_empresas):
  Q420_ = pd.Series(dre3.loc[lista_de_empresas[i],:].loc['Atribuído a Sócios da Empresa Controladora'].iloc[-1] - (
    dre2.loc[lista_de_empresas[i],:].loc['Atribuído a Sócios da Empresa Controladora'].iloc[-5]+
    dre2.loc[lista_de_empresas[i],:].loc['Atribuído a Sócios da Empresa Controladora'].iloc[-2]+
    dre2.loc[lista_de_empresas[i],:].loc['Atribuído a Sócios da Empresa Controladora'].iloc[-1]
))
  Q420 = pd.concat([Q420,Q420_], axis=1)
Q420.columns = lista_de_empresas
Q420

In [ ]:
Q221 = pd.DataFrame()

for i in range(0,n_empresas):
  Q221_ = pd.Series(dre.loc[lista_de_empresas[i],:].loc['Atribuído a Sócios da Empresa Controladora'].iloc[-1])
  Q221 = pd.concat([Q221,Q221_], axis=1)
Q221.columns = lista_de_empresas
Q221

In [ ]:
Q121 = pd.DataFrame()

for i in range(0,n_empresas):
  Q121_ = pd.Series(dre.loc[lista_de_empresas[i],:].loc['Atribuído a Sócios da Empresa Controladora'].iloc[-3])
  Q121 = pd.concat([Q121,Q121_], axis=1)
Q121.columns = lista_de_empresas
Q121

In [ ]:
#ROE
#Junta os lucros líquidos dos últimos 4 trimestre e divide pelo PL ajustado

roe = ((Q320+Q420+Q121+Q221)/pl_ajustado)*100
roe

In [ ]:
fig = go.Figure(go.Bar(
    x=roe.iloc[-1],
    y=tickers,
    orientation='h',text=roe.iloc[-1]
))
fig.update_layout(title_text='Return On Equity')
fig.show()

##5.6. EBIT/Ativo


In [ ]:
#Roic
receita_liq_Q420 = pd.DataFrame()

for i in range(0,n_empresas):
  receita_liq_Q420_ = pd.Series(dre3.loc[lista_de_empresas[i],:].loc['Resultado Antes do Resultado Financeiro e dos Tributos'].iloc[-1] - (
    dre2.loc[lista_de_empresas[i],:].loc['Resultado Antes do Resultado Financeiro e dos Tributos'].iloc[-5]+
    dre2.loc[lista_de_empresas[i],:].loc['Resultado Antes do Resultado Financeiro e dos Tributos'].iloc[-2]+
    dre2.loc[lista_de_empresas[i],:].loc['Resultado Antes do Resultado Financeiro e dos Tributos'].iloc[-1]
))
  receita_liq_Q420 = pd.concat([receita_liq_Q420,receita_liq_Q420_], axis=1)
receita_liq_Q420.columns = lista_de_empresas
receita_liq_Q420

In [ ]:
receita_liq_Q320 = pd.DataFrame()


for i in range(0,n_empresas):
  receita_liq_Q320_ = pd.Series(dre2.loc[lista_de_empresas[i],:].loc['Receita de Venda de Bens e/ou Serviços'].iloc[-1,]+
                             dre2.loc[lista_de_empresas[i],:].loc['Custo dos Bens e/ou Serviços Vendidos'].iloc[-1,]+ 
                             dre2.loc[lista_de_empresas[i],:].loc['Despesas com Vendas'].iloc[-1,]+
                             dre2.loc[lista_de_empresas[i],:].loc['Despesas Gerais e Administrativas'].iloc[-1,])
  receita_liq_Q320 = pd.concat([receita_liq_Q320,receita_liq_Q320_], axis=1)
receita_liq_Q320.columns = lista_de_empresas
receita_liq_Q320




In [ ]:
receita_liq_Q221 = pd.DataFrame()


for i in range(0,n_empresas):
  receita_liq_Q221_ = pd.Series(dre.loc[lista_de_empresas[i],:].loc['Receita de Venda de Bens e/ou Serviços'].iloc[-1,]+
                             dre.loc[lista_de_empresas[i],:].loc['Custo dos Bens e/ou Serviços Vendidos'].iloc[-1,]+ 
                             dre.loc[lista_de_empresas[i],:].loc['Despesas com Vendas'].iloc[-1,]+
                             dre.loc[lista_de_empresas[i],:].loc['Despesas Gerais e Administrativas'].iloc[-1,])
  receita_liq_Q221 = pd.concat([receita_liq_Q221,receita_liq_Q221_], axis=1)
receita_liq_Q221.columns = lista_de_empresas
receita_liq_Q221


In [ ]:
receita_liq_Q121 = pd.DataFrame()


for i in range(0,n_empresas):
  receita_liq_Q121_ = pd.Series(dre.loc[lista_de_empresas[i],:].loc['Receita de Venda de Bens e/ou Serviços'].iloc[-3,]+
                             dre.loc[lista_de_empresas[i],:].loc['Custo dos Bens e/ou Serviços Vendidos'].iloc[-3,]+ 
                             dre.loc[lista_de_empresas[i],:].loc['Despesas com Vendas'].iloc[-3,]+
                             dre.loc[lista_de_empresas[i],:].loc['Despesas Gerais e Administrativas'].iloc[-3,])
  receita_liq_Q121 = pd.concat([receita_liq_Q121,receita_liq_Q121_], axis=1)
receita_liq_Q121.columns = lista_de_empresas
receita_liq_Q121


In [ ]:
receita_liq_Q420 = pd.DataFrame()


for i in range(0,n_empresas):
  receita_liq_Q420_ = pd.Series((dre3.loc[lista_de_empresas[i],:].loc['Receita de Venda de Bens e/ou Serviços'].iloc[-1,]+
                             dre3.loc[lista_de_empresas[i],:].loc['Custo dos Bens e/ou Serviços Vendidos'].iloc[-1,]+ 
                             dre3.loc[lista_de_empresas[i],:].loc['Despesas com Vendas'].iloc[-1,]+
                             dre3.loc[lista_de_empresas[i],:].loc['Despesas Gerais e Administrativas'].iloc[-1,])-
                             ((dre2.loc[lista_de_empresas[i],:].loc['Receita de Venda de Bens e/ou Serviços'].iloc[-1,]+
                             dre2.loc[lista_de_empresas[i],:].loc['Custo dos Bens e/ou Serviços Vendidos'].iloc[-1,]+ 
                             dre2.loc[lista_de_empresas[i],:].loc['Despesas com Vendas'].iloc[-1,]+
                             dre2.loc[lista_de_empresas[i],:].loc['Despesas Gerais e Administrativas'].iloc[-1,])+
                             (dre2.loc[lista_de_empresas[i],:].loc['Receita de Venda de Bens e/ou Serviços'].iloc[-2,]+
                             dre2.loc[lista_de_empresas[i],:].loc['Custo dos Bens e/ou Serviços Vendidos'].iloc[-2,]+ 
                             dre2.loc[lista_de_empresas[i],:].loc['Despesas com Vendas'].iloc[-2,]+
                             (dre2.loc[lista_de_empresas[i],:].loc['Despesas Gerais e Administrativas'].iloc[-5,]))+
                             (dre2.loc[lista_de_empresas[i],:].loc['Receita de Venda de Bens e/ou Serviços'].iloc[-5,]+
                             dre2.loc[lista_de_empresas[i],:].loc['Custo dos Bens e/ou Serviços Vendidos'].iloc[-5,]+ 
                             dre2.loc[lista_de_empresas[i],:].loc['Despesas com Vendas'].iloc[-5,]+
                             dre2.loc[lista_de_empresas[i],:].loc['Despesas Gerais e Administrativas'].iloc[-5,]))
                             )
  receita_liq_Q420 = pd.concat([receita_liq_Q420,receita_liq_Q420_], axis=1)
receita_liq_Q420.columns = lista_de_empresas
receita_liq_Q420

In [ ]:
ebit_ano = receita_liq_Q121+receita_liq_Q221+receita_liq_Q420+receita_liq_Q320
ebit_ano

In [ ]:
#EBIT/ATIVO

ebit_ativo = pd.DataFrame()
for i in range(0,n_empresas):
  ebit_ativo_ = pd.Series(bpa.loc[lista_de_empresas[i],:].loc['Ativo Total'].iloc[-1])
  ebit_ativo = pd.concat([ebit_ativo,ebit_ativo_], axis=1)
ebit_ativo.columns = lista_de_empresas
ebit_ativo = ebit_ano/ebit_ativo
ebit_ativo

fig = go.Figure(go.Bar(
    x=ebit_ativo.iloc[-1],
    y=tickers,
    orientation='h'
))
fig.update_layout(title_text='EBIT/ATIVO')
fig.show()

In [ ]:
bpa.loc[lista_de_empresas[i],:].loc['Caixa e Equivalentes de Caixa'].iloc[-1,] + bpa.loc[lista_de_empresas[i],:].loc['Aplicações Financeiras'].iloc[-1,]

##5.7. ROIC

In [ ]:
roic = ebit_ano/(pl_ajustado+divida_bruta_pl_df+caixa_ajustado)
roic

fig = go.Figure(go.Bar(
    x=roic.iloc[-1],
    y=tickers,
    orientation='h'
))
fig.update_layout(title_text='ROIC')
fig.show()

#6. Equity e Preço

Indicadores fundamentalistas ligados ao preço e valor do ativo


In [ ]:
# Coletar dados de preços dos shoppings
shoppings_preços = pd.DataFrame()

for i in tickers:
  df = investpy.get_stock_historical_data(stock=i, country='Brazil', from_date='31/08/2021', to_date='08/09/2021').Close
  df.rename(i, inplace=True)
  df.columns = [i]
  shoppings_preços = pd.concat([shoppings_preços,df], axis=1)
  shoppings_preços.index.name="Date"

shoppings_preços.columns=lista_de_empresas
shoppings_preços

In [ ]:
#Número de açoes da empresa

n_acoes = pd.DataFrame()

for i in tickers:
  df_ =investpy.get_stock_information(i, country='Brazil')['Shares Outstanding']
  df_.rename(i,inplace=True)
  df_.columns = [i]
  n_acoes = pd.concat([n_acoes,df_], axis=1)
  
n_acoes.columns=lista_de_empresas
n_acoes

In [ ]:
n_acoes = n_acoes/1000
n_acoes

##6.1. Valor de Mercado

In [ ]:
#MarketCap das ações

marketcap = n_acoes*shoppings_preços.iloc[-1]
marketcap

In [ ]:
fig = go.Figure(go.Bar(
    x=marketcap.iloc[-1],
    y=tickers,
    orientation='h',text=marketcap.iloc[-1]
))
fig.update_layout(title_text='Marketcap')
fig.show()

##6.2. Valor da Firma

In [ ]:
#Enterprise Value

ev = marketcap + (divida_bruta_pl_df-caixa_ajustado)

In [ ]:
(divida_bruta_pl_df-caixa_ajustado)

In [ ]:
ev

fig = go.Figure(go.Bar(
    x=ev.iloc[-1],
    y=tickers,
    orientation='h'
))
fig.update_layout(title_text='Enterprise Value')
fig.show()

##6.3. Valor Patrimonial da Ação

In [ ]:
#Valor Patrimonial da Ação

vpa = (pl_ajustado)/n_acoes

In [ ]:
vpa

In [ ]:
fig = go.Figure(go.Bar(
    x=vpa.iloc[-1],
    y=tickers,
    orientation='h'
))
fig.update_layout(title_text='Valor Patrimonial da Ação')
fig.show()

In [ ]:
type(pl_ajustado.iloc[-1,-1])

##6.4. Lucro por Ação e P/L

In [ ]:
#LPA

lpa = (Q320+Q420+Q121+Q221)/n_acoes
lpa

In [ ]:
#P/L

pl = shoppings_preços.iloc[-1]/lpa
pl

In [ ]:
fig = go.Figure(go.Bar(
    x=pl.iloc[-1],
    y=tickers,
    orientation='h',
))
fig.update_layout(title_text='Preço/Lucro')
fig.show()

In [ ]:
#P/VPA
pvpa = shoppings_preços.iloc[-1]/vpa
pvpa

In [ ]:


fig = go.Figure(go.Bar(
    x=pvpa.iloc[-1],
    y=tickers,
    orientation='h',text=pvpa.iloc[-1]
))
fig.update_layout(title_text='P/VPA')
fig.show()

In [ ]:

fig = ff.create_table(lpa, height_constant=50)
trace1=go.Bar(x=tickers, y=pl.iloc[-1],marker=dict(color='#0099ff'), name='P/L',xaxis='x2' ,yaxis='y2')
fig.add_traces([trace1])
# initialize xaxis2 and yaxis2
fig['layout']['xaxis2'] = {}
fig['layout']['yaxis2'] = {}

# Edit layout for subplots
fig.layout.yaxis.update({'domain': [0, .45]})
fig.layout.yaxis2.update({'domain': [.5, 1]})
fig.layout.xaxis2.update({'anchor': 'y2'})
fig.layout.yaxis2.update({'title': 'Goals'})

fig.layout.margin.update({'t':75, 'l':50})
fig.layout.update({'title': 'Analitcs'})

# Update the height because adding a graph vertically will interact with
# the plot height calculated for the table
fig.layout.update({'height':800})


fig.show()